# Compare different classifiers using HT29 dataset ( 14 phenotypes) in a binary fashion (each phenotype has its own data)

## Obtain Data from database and only choose the positive examples to be in training set

In [ ]:
#modify files and put them in another directory
import os
import numpy as np
import csv
directory = '/Users/jyhung/Downloads/SupplementalData/Jones_PNAS_2009_training_sets/'
newDirectory = '/Users/jyhung/work/projects/cpamulti/Jones_PNAS_2009/'
#
def getNumlabel(strlabel):
    if strlabel in label_to_labelidx:
        return label_to_labelidx[strlabel]
    global curlabel
    print "LABEL: ", curlabel, strlabel
    label_to_labelidx[strlabel] = curlabel
    curlabel += 1
    return label_to_labelidx[strlabel]

for dirname, dirnames, filenames in os.walk(directory):

    label_names = np.array([])
    labels = []
    label_to_labelidx = {}
    curlabel = 1
    idNumbers = [] #each entry is (image number, object number)
    # print path to all filenames.
    #for each file, take out the lines that start with +1 
    #and add each to the label array to count the number in each class
    for filename in filenames:
        #filename_bare, file_extension = os.path.splitext(filename)
        #if file_extension == '.txt':
        #    print(filename_bare)#print(os.path.join(dirname, filename))
        if filename.endswith('_training.txt'):
            #print filename
            label_name = filename.split('_training.txt')[0]
            label_names = np.append(label_names, label_name)
            numlabel = getNumlabel(label_name)
            
            reader = csv.reader(open(os.path.join(directory,filename)), delimiter=' ')
            header = reader.next()
            print header
            with open(os.path.join(newDirectory,filename.split('.txt')[0] + '_modified.txt'), 'wb') as csvfile:
                
                writer = csv.writer(csvfile, delimiter=' ',quotechar=' ', quoting=csv.QUOTE_MINIMAL)
                writer.writerow([header[0], header[2], header [3]])
                for vals in reader:
                    #idNumbers.append()
                    #print vals
                    writer.writerow([vals[0], vals[2], vals[3]])
                

In [ ]:
import os
import numpy as np
import csv
directory = '/Users/jyhung/Downloads/SupplementalData/Jones_PNAS_2009_training_sets/'

#
def getNumlabel(strlabel):
    if strlabel in label_to_labelidx:
        return label_to_labelidx[strlabel]
    global curlabel
    print "LABEL: ", curlabel, strlabel
    label_to_labelidx[strlabel] = curlabel
    curlabel += 1
    return label_to_labelidx[strlabel]

for dirname, dirnames, filenames in os.walk(directory):

    label_names = np.array([])
    labels = []
    label_to_labelidx = {}
    curlabel = 1
    idNumbers = [] #each entry is (image number, object number)
    # print path to all filenames.
    #for each file, take out the lines that start with +1 
    #and add each to the label array to count the number in each class
    for filename in filenames:
        #filename_bare, file_extension = os.path.splitext(filename)
        #if file_extension == '.txt':
        #    print(filename_bare)#print(os.path.join(dirname, filename))
        if filename.endswith('actin_blebs_training.txt'):
            #print filename
            label_name = filename.split('_training.txt')[0]
            label_names = np.append(label_names, label_name)
            numlabel = getNumlabel(label_name)
            
            reader = csv.reader(open(os.path.join(directory,filename)), delimiter=' ')
            header = reader.next()
            print header
            for vals in reader:
                #idNumbers.append()
                print vals
                if vals[0] == '+1':
                    labels.append(numlabel)
                    idNumbers.append((int(vals[2]), int(vals[3])))
                    print labels, idNumbers

    
            
print label_names, len(label_names)



remove duplicates (samples with more than 1 phenotype) "ambiguously classified cells"

In [ ]:
from collections import defaultdict
D = defaultdict(list)
for i,item in enumerate(idNumbers):
    D[item].append(i)

D = {k:v for k,v in D.items() if len(v)>1}
print D
duplicate_indices = np.array([])
for keys in D:
    duplicate_indices = np.append(duplicate_indices, D[keys])
    
print duplicate_indices
idNumbers_1occurance = [i for j, i in enumerate(idNumbers) if j not in duplicate_indices]
print idNumbers_1occurance

get data from database

In [ ]:
#function to turn multiclass labels into binary 
def multiclassToBinary(class_one, labels):
    '''
    class_one: class of interest
    labels: array of class number of each sample
    RETURNS: modified labels array where class_one is the positive class and all other classes are in the negative class
    '''
    return [1 if x==class_one else 0 for x in labels]

In [ ]:
#test, example
import sys
sys.path.insert(0, '/Users/jyhung/Documents/CellProfiler-Analyst/cpa/')
from dbconnect import *
import threading
from trainingset import TrainingSet
from datamodel import DataModel
from trainingset import TrainingSet
from StringIO import StringIO
import generalclassifier
from datatable import DataGrid
from sklearn.ensemble import AdaBoostClassifier

import wx
p = Properties.getInstance()
db = DBConnect.getInstance()
dm = DataModel.getInstance()

props = '/Users/jyhung/work/projects/subpop/analysis/input/2013-01-16_Handtraining_using_only_the_DNA_stain/training_sets/2013_01_17_Anne_AZ_DNA_training/az-dnaonly.properties'
ts = '/Users/jyhung/work/projects/subpop/analysis/input/2013-01-16_Handtraining_using_only_the_DNA_stain/training_sets/2013_01_17_Anne_AZ_DNA_training/Anne_DNA_66.txt'
nRules = 5
filter = None

classifier = AdaBoostClassifier(n_estimators=nRules)
GC = generalclassifier.GeneralClassifier(classifier)

p.LoadFile(props)
trainingSet = TrainingSet(p)
trainingSet.Load(ts)
print trainingSet.labels
print len(trainingSet.colnames)
print trainingSet.colnames
print trainingSet.values.shape
print trainingSet.values

In [ ]:
import sys
sys.path.insert(0, '/Users/jyhung/Documents/CellProfiler-Analyst/cpa/')
from dbconnect import *
import threading
from trainingset import TrainingSet

def get_object_data(object_table, key):
    self.data[key] = db.GetCellData(key)
    return self.data[key][self.col_indices]

p = Properties.getInstance()
db = DBConnect.getInstance()
props ='/Users/jyhung/work/projects/cpamulti/Jones_PNAS_2009/nirht.properties'
ts='/Users/jyhung/work/projects/cpamulti/Jones_PNAS_2009/actin_blebs_training_modified.txt'
p.LoadFile(props)
trainingSet = TrainingSet(p)
trainingSet.Load(ts)



In [ ]:
! grep classifier_ignore_columns /Users/jyhung/work/projects/cpamulti/Jones_PNAS_2009/nirht.properties

In [ ]:
trainingSet.values.shape
values = trainingSet.values
labels= trainingSet.label_matrix[:,2]

In [ ]:
#print trainingSet.values.shape
#print trainingSet.colnames
#print trainingSet.values, trainingSet.values.shape
#print trainingSet.label_matrix[:,2]

In [ ]:
import sys
sys.path.insert(0, '/Users/jyhung/Documents/CellProfiler-Analyst/cpa/')
import generalclassifier, multiclasssql
def XValidateBinary(classifier, labels, values):
    classifier = generalclassifier.GeneralClassifier(eval(classifier))
    unique_labels, inversed = np.unique(labels, return_inverse=True)
    num_classes = len(unique_labels) 
    for i in range(1, num_classes+1):
        labels_binary = multiclassToBinary(i, labels)
        scores = classifier.XValidate(labels_binary, values, folds=20, stratified=True)
        print scores.mean(), scores.std()

In [ ]:
sum(labels==1)

In [ ]:
unique_labels = np.unique(labels)
n_classes = len(unique_labels)
scorePerClass = np.zeros(n_classes)
predicted_labels = generalclassifier.GeneralClassifier(eval(classifier)).XValidatePredict(labels,values,folds=20, stratified=True)
print predicted_labels
for i in unique_labels:
    indices = (labels == (i))
    print i, sum(indices)
    #print predicted_labels[indices] == (i)
    predicted_correct = (predicted_labels[indices] == (i))
    print 1.0*sum(predicted_correct)/sum(indices)

In [3]:
#run to save training data (label and values) in npz files
import os
import sys
sys.path.insert(1, '/Users/jyhung/Documents/CellProfiler-Analyst/cpa/')
sys.path.insert(1, '/Users/jyhung/Documents/CellProfiler-Analyst/')
from dbconnect import *
import threading
from trainingset import TrainingSet
import generalclassifier, multiclasssql
from sklearn import ensemble, naive_bayes, grid_search, svm, lda, qda, tree, multiclass, linear_model, neighbors

directory = '/Users/jyhung/work/projects/cpamulti/Jones_PNAS_2009/'
p = Properties.getInstance()
db = DBConnect.getInstance()
props =directory+'nirht.properties'
p.LoadFile(props)
trainingSet = TrainingSet(p)
for dirname, dirnames, filenames in os.walk(directory):
        for filename in sorted(filenames):

            if filename.endswith('_modified.txt'):
                ts=directory+filename
                print 'Loading training set %s ...' % ts
                trainingSet.Load(ts)
                print 'Accessing training set ...'
                values = trainingSet.values
                labels= trainingSet.label_matrix[:,2]
                labels = np.array([1 if x==1 else 2 for x in labels])#1 for positives, 2 for negatives
                np.savez(directory+filename.split('_modified.txt')[0], labels=labels, values=values)
                #np.save(directory+filename.split('_modified.txt')[0]+'_values', values)


Loading training set /Users/jyhung/work/projects/cpamulti/Jones_PNAS_2009/actin_blebs_training_modified.txt ...
Accessing training set ...
Loading training set /Users/jyhung/work/projects/cpamulti/Jones_PNAS_2009/actin_dots_junctions_training_modified.txt ...
Accessing training set ...
Loading training set /Users/jyhung/work/projects/cpamulti/Jones_PNAS_2009/anaphase_telophase_training_modified.txt ...
Accessing training set ...
Loading training set /Users/jyhung/work/projects/cpamulti/Jones_PNAS_2009/angular_cell_edges_training_modified.txt ...
Accessing training set ...
Loading training set /Users/jyhung/work/projects/cpamulti/Jones_PNAS_2009/crescent_nuclei_training_modified.txt ...
Accessing training set ...
Loading training set /Users/jyhung/work/projects/cpamulti/Jones_PNAS_2009/large_spread_training_modified.txt ...
Accessing training set ...
Loading training set /Users/jyhung/work/projects/cpamulti/Jones_PNAS_2009/long_projections_training_modified.txt ...
Accessing training se

## Training for 1 phenotype

In [ ]:
import sys
sys.path.insert(0, '/Users/jyhung/Documents/CellProfiler-Analyst/cpa/')
from dbconnect import *
import threading
from trainingset import TrainingSet

p = Properties.getInstance()
db = DBConnect.getInstance()
props ='/Users/jyhung/work/projects/cpamulti/Jones_PNAS_2009/nirht.properties'
ts='/Users/jyhung/work/projects/cpamulti/Jones_PNAS_2009/actin_blebs_training_modified.txt'
p.LoadFile(props)
trainingSet = TrainingSet(p)
trainingSet.Load(ts)

values = trainingSet.values
labels= trainingSet.label_matrix[:,2]

In [ ]:
import sys
sys.path.insert(0, '/Users/jyhung/Documents/CellProfiler-Analyst/cpa/')
import generalclassifier, multiclasssql
from sklearn import ensemble, naive_bayes, grid_search, svm, lda, qda, tree, multiclass, linear_model, neighbors

classifier = 'ensemble.AdaBoostClassifier()'
def main(classifier, labels_, values_):
    classifier = generalclassifier.GeneralClassifier(eval(classifier))
    scores = classifier.XValidate(labels_, values_, folds=20, stratified=True)
    print scores.mean(), scores.std()

    print classifier.ClassificationReport(labels_, classifier.XValidatePredict(labels_,values_,folds=20, stratified=True))
main(classifier, labels, values)

## Training for 14 phenotypes

In [1]:
#1 run to define tic and toc (a la matlab)
import time
def tic():
    #Homemade version of matlab tic and toc functions
    global startTime_for_tictoc
    startTime_for_tictoc = time.time()

def toc():
    if 'startTime_for_tictoc' in globals():
        print "Elapsed time is " + str(time.time() - startTime_for_tictoc) + " seconds."
    else:
        print "Toc: start time not set"

In [1]:
#2 run to define main function which gets training data from database and does CV
import os
import sys
sys.path.insert(1, '/Users/jyhung/Documents/CellProfiler-Analyst/cpa/')
sys.path.insert(1, '/Users/jyhung/Documents/CellProfiler-Analyst/')
from dbconnect import *
import threading
from trainingset import TrainingSet
import generalclassifier, multiclasssql
from sklearn import ensemble, naive_bayes, grid_search, svm, lda, qda, tree, multiclass, linear_model, neighbors

directory = '/Users/jyhung/work/projects/cpamulti/Jones_PNAS_2009/'
p = Properties.getInstance()
db = DBConnect.getInstance()
props =directory+'nirht.properties'
p.LoadFile(props)
trainingSet = TrainingSet(p)

def main(classifier_obj):
    classifier = generalclassifier.GeneralClassifier(eval(classifier_obj))
    
    for dirname, dirnames, filenames in os.walk(directory):
        for filename in sorted(filenames):

            if filename.endswith('_modified.txt'):
                ts=directory+filename
                print 'Loading training set %s ...' % ts
                trainingSet.Load(ts)
                print 'Accessing training set ...'
                values = trainingSet.values
                labels= trainingSet.label_matrix[:,2]
                labels = np.array([1 if x==1 else 2 for x in labels])
                np.save(filename.split('_modified.txt')[0]+'_labels.txt', labels)
                np.save(filename.split('_modified.txt')[0]+'_values.txt', values)

                print filename, values.shape
                #print labels, values
                #%time classifier.XValidate(labels, values, folds=20, stratified=True)
                tic()
                print 'Scoring cells 1 ...'
                scores = classifier.XValidate(labels, values, folds=20, stratified=True)
                toc()
                print 'stratified 20 fold CV', scores.mean(), scores.std()
                #print 'Scoring cells 2 ...'
                #scores2 = classifier.XValidate(labels, values, folds=5, stratified=True)
                #print 'stratified 5 fold CV', scores2.mean(), scores2.std()
                '''
                try:
                    scores3 = classifier.XValidateBalancedClasses(labels, values, folds=5)
                    print 'undersampled (balanced)', scores3.mean(), scores3.std()
                except:
                    pass
                '''
                print classifier.ClassificationReport(labels, classifier.XValidatePredict(labels,values,folds=20, stratified=True))




In [2]:
#2 run to define main function which loads training data from local .npy files and does CV
import os
import sys
sys.path.insert(1, '/Users/jyhung/Documents/CellProfiler-Analyst/cpa/')
sys.path.insert(1, '/Users/jyhung/Documents/CellProfiler-Analyst/')
from dbconnect import *
import threading
from trainingset import TrainingSet
import generalclassifier, multiclasssql
from sklearn import cross_validation
from sklearn import ensemble, naive_bayes, grid_search, svm, lda, qda, tree, multiclass, linear_model, neighbors

directory = '/Users/jyhung/work/projects/cpamulti/Jones_PNAS_2009/'

def main(classifier_obj):
    classifier = generalclassifier.GeneralClassifier(eval(classifier_obj))
    
    for dirname, dirnames, filenames in os.walk(directory):
        for filename in sorted(filenames):
            
            if filename.endswith('.npz'):
                print 'Loading training set ...'
                trainingSet=np.load(directory+filename)
                labels=trainingSet['labels']
                values=trainingSet['values']
                print filename, values.shape
                #print labels, values
                #%time classifier.XValidate(labels, values, folds=20, stratified=True)
                tic()
                print 'Scoring cells 1 ...'
                scores = classifier.XValidate(labels, values, folds=5, stratified=True)
                toc()
                print 'stratified 5 fold CV', scores.mean(), scores.std()
                print classifier.ClassificationReport(labels, classifier.XValidatePredict(labels,values,folds=5, stratified=True))




In [5]:
import numpy as np
#ddd=np.load(directory+'angular_cell_edges_training.npz')
ddd=np.load(directory+'anaphase_telophase_training.npz')
labels = ddd['labels']
values = ddd['values']
classifier_obj = "grid_search.GridSearchCV(svm.LinearSVC(),param_grid={'C': [2**-5, 2**-4, 2**-3, 2**-2, 2**-1, 2**0, 2**1, 2**2, 2**3, 2**4, 2**5, 2**6]},cv=5,n_jobs=1)"
#classifier = generalclassifier.GeneralClassifier(eval(classifier_obj))
#print classifier.XValidate(labels, values, folds=20, stratified=True)
print cross_validation.cross_val_score(eval(classifier_obj), X=values, y=labels, cv=20)


[ 0.93548387  0.93548387  0.98924731  0.96774194  0.88172043  0.94623656
  0.86021505  0.97826087  0.84782609  0.97826087  0.95652174  0.97826087
  0.95652174  0.97802198  0.92307692  0.94505495  0.98901099  0.96703297
  0.95604396  0.78021978]


In [ ]:
import numpy as np
#ddd=np.load(directory+'angular_cell_edges_training.npz')
ddd=np.load(directory+'anaphase_telophase_training.npz')
labels = ddd['labels']
values = ddd['values']
print labels, values
classifier_obj = "grid_search.GridSearchCV(svm.LinearSVC(),param_grid={'C': [2**-5, 2**-4, 2**-3, 2**-2, 2**-1, 2**0, 2**1, 2**2, 2**3, 2**4, 2**5, 2**6]},cv=5,n_jobs=1)"
#classifier = generalclassifier.GeneralClassifier(eval(classifier_obj))
#print classifier.XValidate(labels, values, folds=20, stratified=True)
print cross_validation.cross_val_score(eval(classifier_obj), X=values, y=labels, cv=20, n_jobs=1) #when n_jobs !=1, hangs

## AdaBoost

In [57]:
#Adaboost 50
classifier = 'ensemble.AdaBoostClassifier()'
main(classifier)

Loading training set ...
actin_blebs_training.npz (509, 617)
Scoring cells 1 ...
Elapsed time is 5.6129360199 seconds.
stratified 5 fold CV 0.829004076878 0.0422210215265
             precision    recall  f1-score   support

          1       0.67      0.48      0.56       115
          2       0.86      0.93      0.89       394

avg / total       0.82      0.83      0.82       509

Loading training set ...
actin_dots_junctions_training.npz (302, 617)
Scoring cells 1 ...
Elapsed time is 3.39117312431 seconds.
stratified 5 fold CV 0.791092896175 0.0636558312892
             precision    recall  f1-score   support

          1       0.73      0.76      0.74       120
          2       0.84      0.81      0.82       182

avg / total       0.79      0.79      0.79       302

Loading training set ...
anaphase_telophase_training.npz (1840, 617)
Scoring cells 1 ...
Elapsed time is 21.7887527943 seconds.
stratified 5 fold CV 0.947356090209 0.041291594856
             precision    recall  f1-sc

## Gradient Boosting

In [59]:
#gradient boost 100
classifier = 'ensemble.GradientBoostingClassifier()'
main(classifier)

Loading training set ...
actin_blebs_training.npz (509, 617)
Scoring cells 1 ...
Elapsed time is 17.8596830368 seconds.
stratified 5 fold CV 0.821160939623 0.0411295113369
             precision    recall  f1-score   support

          1       0.70      0.37      0.49       115
          2       0.84      0.95      0.89       394

avg / total       0.81      0.82      0.80       509

Loading training set ...
actin_dots_junctions_training.npz (302, 617)
Scoring cells 1 ...
Elapsed time is 10.4773581028 seconds.
stratified 5 fold CV 0.81087431694 0.0708750542407
             precision    recall  f1-score   support

          1       0.75      0.79      0.77       120
          2       0.86      0.82      0.84       182

avg / total       0.81      0.81      0.81       302

Loading training set ...
anaphase_telophase_training.npz (1840, 617)
Scoring cells 1 ...
Elapsed time is 65.1136770248 seconds.
stratified 5 fold CV 0.966348436734 0.0209346550405
             precision    recall  f1-s

## SVM

In [4]:
#1 vs all linear svm
classifier = "grid_search.GridSearchCV(svm.LinearSVC(),param_grid={'C': [2**-5, 2**-4, 2**-3, 2**-2, 2**-1, 2**0, 2**1, 2**2, 2**3, 2**4, 2**5, 2**6]},cv=5,n_jobs=1)"
main(classifier)

Loading training set ...
actin_blebs_training.npz (509, 617)
Scoring cells 1 ...
Elapsed time is 88.0731480122 seconds.
stratified 5 fold CV 0.707493690546 0.0794806610694
             precision    recall  f1-score   support

          1       0.40      0.50      0.45       115
          2       0.84      0.79      0.81       394

avg / total       0.74      0.72      0.73       509

Loading training set ...
actin_dots_junctions_training.npz (302, 617)
Scoring cells 1 ...
Elapsed time is 54.7562119961 seconds.
stratified 5 fold CV 0.711420765027 0.0819909619512
             precision    recall  f1-score   support

          1       0.61      0.70      0.65       120
          2       0.78      0.70      0.74       182

avg / total       0.71      0.70      0.70       302

Loading training set ...
anaphase_telophase_training.npz (1840, 617)
Scoring cells 1 ...
Elapsed time is 184.704161167 seconds.
stratified 5 fold CV 0.939166942774 0.0257599854582
             precision    recall  f1-

KeyboardInterrupt: 

In [3]:
#1 vs all linear svm, auto weights 
classifier = "grid_search.GridSearchCV(svm.LinearSVC(penalty='l2', class_weight='auto'),param_grid={'C': [2**-5, 2**-4, 2**-3, 2**-2, 2**-1, 2**0, 2**1, 2**2, 2**3, 2**4, 2**5, 2**6]},cv=5,n_jobs=1)"
main(classifier)

Loading training set ...
actin_blebs_training.npz (509, 617)
Scoring cells 1 ...
Elapsed time is 84.4147870541 seconds.
stratified 5 fold CV 0.738827412153 0.0729213429824
             precision    recall  f1-score   support

          1       0.46      0.48      0.47       115
          2       0.85      0.84      0.84       394

avg / total       0.76      0.75      0.76       509

Loading training set ...
actin_dots_junctions_training.npz (302, 617)
Scoring cells 1 ...
Elapsed time is 55.7725188732 seconds.
stratified 5 fold CV 0.737595628415 0.100795693685
             precision    recall  f1-score   support

          1       0.62      0.62      0.62       120
          2       0.75      0.75      0.75       182

avg / total       0.70      0.70      0.70       302

Loading training set ...
anaphase_telophase_training.npz (1840, 617)
Scoring cells 1 ...
Elapsed time is 164.990232944 seconds.
stratified 5 fold CV 0.931016225649 0.030718158007
             precision    recall  f1-sc

In [68]:
#1 vs all linear svm, auto weights, no search
classifier = "svm.LinearSVC(penalty='l2', class_weight='auto')"
main(classifier)

Loading training set ...
actin_blebs_training.npz (509, 617)
Scoring cells 1 ...
Elapsed time is 1.77711701393 seconds.
stratified 5 fold CV 0.774102116094 0.0427799151412
             precision    recall  f1-score   support

          1       0.36      0.49      0.41       115
          2       0.83      0.75      0.79       394

avg / total       0.73      0.69      0.70       509

Loading training set ...
actin_dots_junctions_training.npz (302, 617)
Scoring cells 1 ...
Elapsed time is 1.03716421127 seconds.
stratified 5 fold CV 0.688415300546 0.0533515265037
             precision    recall  f1-score   support

          1       0.63      0.69      0.66       120
          2       0.78      0.73      0.76       182

avg / total       0.72      0.72      0.72       302

Loading training set ...
anaphase_telophase_training.npz (1840, 617)
Scoring cells 1 ...
Elapsed time is 3.76545786858 seconds.
stratified 5 fold CV 0.915332008017 0.0635373404695
             precision    recall  f1-

In [ ]:
#1 vs all linear svm, l1 penalty, auto weights 
classifier = "grid_search.GridSearchCV(svm.LinearSVC(penalty='l1',dual=False, class_weight='auto'),param_grid={'C': [2**-5, 2**-4, 2**-3, 2**-2, 2**-1, 2**0, 2**1, 2**2, 2**3, 2**4, 2**5, 2**6]},cv=5,n_jobs=1)"
main(classifier)

Loading training set ...
actin_blebs_training.npz (509, 617)
Scoring cells 1 ...
Elapsed time is 80.5314002037 seconds.
stratified 5 fold CV 0.766181324015 0.0743345558018
             precision    recall  f1-score   support

          1       0.53      0.54      0.54       115
          2       0.87      0.86      0.86       394

avg / total       0.79      0.79      0.79       509

Loading training set ...
actin_dots_junctions_training.npz (302, 617)
Scoring cells 1 ...
Elapsed time is 46.1349279881 seconds.
stratified 5 fold CV 0.764535519126 0.0591993722795
             precision    recall  f1-score   support

          1       0.68      0.79      0.73       120
          2       0.85      0.75      0.80       182

avg / total       0.78      0.77      0.77       302

Loading training set ...
anaphase_telophase_training.npz (1840, 617)
Scoring cells 1 ...
Elapsed time is 351.477404833 seconds.
stratified 5 fold CV 0.918538262077 0.0428188436378
             precision    recall  f1-

In [ ]:
#1 vs 1 linear svm
classifier = "grid_search.GridSearchCV(svm.SVC(kernel='linear'),param_grid={'C': [2**-5, 2**-4, 2**-3, 2**-2, 2**-1, 2**0, 2**1, 2**2, 2**3, 2**4, 2**5, 2**6]},cv=5,n_jobs=1)"
main(classifier)

Loading training set ...
actin_blebs_training.npz (509, 617)
Scoring cells 1 ...
Elapsed time is 357.650186777 seconds.
stratified 5 fold CV 0.738652688798 0.0299442555719
             precision    recall  f1-score   support

          1       0.43      0.50      0.46       115
          2       0.85      0.81      0.83       394

avg / total       0.75      0.74      0.74       509

Loading training set ...
actin_dots_junctions_training.npz (302, 617)
Scoring cells 1 ...
Elapsed time is 82.8608181477 seconds.
stratified 5 fold CV 0.675245901639 0.052542226811
             precision    recall  f1-score   support

          1       0.59      0.62      0.60       120
          2       0.74      0.71      0.72       182

avg / total       0.68      0.68      0.68       302

Loading training set ...
anaphase_telophase_training.npz (1840, 617)
Scoring cells 1 ...
Elapsed time is 779.299878836 seconds.
stratified 5 fold CV 0.922878772118 0.023972855571
             precision    recall  f1-sc

In [60]:
#svm rbf
classifier = "grid_search.GridSearchCV(svm.SVC(),param_grid={'gamma': [2**-5, 2**-4, 2**-3, 2**-2, 2**-1, 2**0, 2**1], 'C': [2**-1, 2**0, 2**1, 2**2, 2**3, 2**4, 2**5, 2**6]},cv=5,n_jobs=-1)"
main(classifier)

Loading training set ...
actin_blebs_training.npz (509, 617)
Scoring cells 1 ...
Elapsed time is 28.1089689732 seconds.
stratified 5 fold CV 0.774063288682 0.000893030479519
             precision    recall  f1-score   support

          1       0.00      0.00      0.00       115
          2       0.77      1.00      0.87       394

avg / total       0.60      0.77      0.68       509

Loading training set ...
actin_dots_junctions_training.npz (302, 617)
Scoring cells 1 ...
Elapsed time is 11.5640640259 seconds.
stratified 5 fold CV 0.605901639344 0.00889485899426
             precision    recall  f1-score   support

          1       1.00      0.01      0.02       120
          2       0.60      1.00      0.75       182

avg / total       0.76      0.61      0.46       302

Loading training set ...
anaphase_telophase_training.npz (1840, 617)
Scoring cells 1 ...
Elapsed time is 346.586475849 seconds.
stratified 5 fold CV 0.898371916947 0.00110919664361


KeyboardInterrupt: 

In [61]:
#svm rbf, class weight auto
classifier = "grid_search.GridSearchCV(svm.SVC(class_weight='auto'),param_grid={'gamma': [2**-5, 2**-4, 2**-3, 2**-2, 2**-1, 2**0, 2**1], 'C': [2**-1, 2**0, 2**1, 2**2, 2**3, 2**4, 2**5, 2**6]},cv=5,n_jobs=-1)"
main(classifier)

Loading training set ...
actin_blebs_training.npz (509, 617)
Scoring cells 1 ...
Elapsed time is 27.4059941769 seconds.
stratified 5 fold CV 0.781906425937 0.0100036926712
             precision    recall  f1-score   support

          1       1.00      0.03      0.07       115
          2       0.78      1.00      0.88       394

avg / total       0.83      0.78      0.69       509

Loading training set ...
actin_dots_junctions_training.npz (302, 617)
Scoring cells 1 ...
Elapsed time is 10.8943209648 seconds.
stratified 5 fold CV 0.632131147541 0.0396924483945
             precision    recall  f1-score   support

          1       1.00      0.07      0.14       120
          2       0.62      1.00      0.77       182

avg / total       0.77      0.63      0.52       302

Loading training set ...
anaphase_telophase_training.npz (1840, 617)
Scoring cells 1 ...
Elapsed time is 315.378823042 seconds.
stratified 5 fold CV 0.901641671715 0.00475062923119
             precision    recall  f1

KeyboardInterrupt: 

## LDA

In [3]:
classifier = "lda.LDA()"
main(classifier)

Loading training set ...
actin_blebs_training.npz (509, 617)
Scoring cells 1 ...
Elapsed time is 0.418704032898 seconds.
stratified 5 fold CV 0.62686856921 0.0790804157202
             precision    recall  f1-score   support

          1       0.31      0.55      0.40       115
          2       0.83      0.65      0.73       394

avg / total       0.71      0.63      0.65       509

Loading training set ...
actin_dots_junctions_training.npz (302, 617)
Scoring cells 1 ...
Elapsed time is 0.186514854431 seconds.
stratified 5 fold CV 0.695136612022 0.0742358123566
             precision    recall  f1-score   support

          1       0.60      0.72      0.65       120
          2       0.79      0.68      0.73       182

avg / total       0.71      0.70      0.70       302

Loading training set ...
anaphase_telophase_training.npz (1840, 617)
Scoring cells 1 ...
Elapsed time is 1.2811319828 seconds.
stratified 5 fold CV 0.905567129596 0.0852560247984
             precision    recall  f1-

/Users/jyhung/anaconda/lib/python2.7/site-packages/sklearn/lda.py:371: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


## Random Forest

In [5]:
#random forest 100
classifier = "ensemble.RandomForestClassifier(n_estimators=100)"
main(classifier)

Loading training set ...
actin_blebs_training.npz (509, 617)
Scoring cells 1 ...
Elapsed time is 1.72977089882 seconds.
stratified 5 fold CV 0.836944282664 0.0219353745913
             precision    recall  f1-score   support

          1       0.89      0.37      0.52       115
          2       0.84      0.99      0.91       394

avg / total       0.85      0.85      0.82       509

Loading training set ...
actin_dots_junctions_training.npz (302, 617)
Scoring cells 1 ...
Elapsed time is 0.937800884247 seconds.
stratified 5 fold CV 0.827322404372 0.0831188676262
             precision    recall  f1-score   support

          1       0.74      0.75      0.75       120
          2       0.83      0.83      0.83       182

avg / total       0.80      0.80      0.80       302

Loading training set ...
anaphase_telophase_training.npz (1840, 617)
Scoring cells 1 ...
Elapsed time is 7.35504484177 seconds.
stratified 5 fold CV 0.970130651816 0.0117144102093
             precision    recall  f1

In [4]:
#random forest 100, class weights auto
classifier = "ensemble.RandomForestClassifier(n_estimators=100, class_weight='auto')"
main(classifier)

Loading training set ...
actin_blebs_training.npz (509, 617)
Scoring cells 1 ...
Elapsed time is 1.68912696838 seconds.
stratified 5 fold CV 0.842787808193 0.0168320974118
             precision    recall  f1-score   support

          1       0.88      0.38      0.53       115
          2       0.85      0.98      0.91       394

avg / total       0.85      0.85      0.82       509

Loading training set ...
actin_dots_junctions_training.npz (302, 617)
Scoring cells 1 ...
Elapsed time is 0.984733819962 seconds.
stratified 5 fold CV 0.817540983607 0.0747417143738
             precision    recall  f1-score   support

          1       0.76      0.80      0.78       120
          2       0.86      0.83      0.85       182

avg / total       0.82      0.82      0.82       302

Loading training set ...
anaphase_telophase_training.npz (1840, 617)
Scoring cells 1 ...
Elapsed time is 5.43034505844 seconds.
stratified 5 fold CV 0.972303116098 0.0101726215612
             precision    recall  f1

/Users/jyhung/anaconda/lib/python2.7/site-packages/sklearn/metrics/classification.py:958: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [ ]:
#random forest 1000
classifier = "ensemble.RandomForestClassifier(n_estimators=1000)"
main(classifier)

## Naive Bayes

In [6]:
classifier = "naive_bayes.GaussianNB()"
main(classifier)

Loading training set ...
actin_blebs_training.npz (509, 617)
Scoring cells 1 ...
Elapsed time is 0.0246541500092 seconds.
stratified 5 fold CV 0.597185012619 0.0685946265532
             precision    recall  f1-score   support

          1       0.34      0.87      0.49       115
          2       0.93      0.52      0.67       394

avg / total       0.80      0.60      0.63       509

Loading training set ...
actin_dots_junctions_training.npz (302, 617)
Scoring cells 1 ...
Elapsed time is 0.0216488838196 seconds.
stratified 5 fold CV 0.727704918033 0.136536547457
             precision    recall  f1-score   support

          1       0.61      0.88      0.72       120
          2       0.88      0.63      0.74       182

avg / total       0.78      0.73      0.73       302

Loading training set ...
anaphase_telophase_training.npz (1840, 617)
Scoring cells 1 ...
Elapsed time is 0.101501941681 seconds.
stratified 5 fold CV 0.52404855286 0.208300586246
             precision    recall  f

## Logistic Regression

In [63]:
#logistic regression 1 vs all, no search
classifier = "linear_model.LogisticRegression(penalty='l2',solver='liblinear', multi_class='ovr', class_weight=None)" 
main(classifier)

Loading training set ...
actin_blebs_training.npz (509, 617)
Scoring cells 1 ...
Elapsed time is 2.95930504799 seconds.
stratified 5 fold CV 0.750495049505 0.0252061835998
             precision    recall  f1-score   support

          1       0.46      0.55      0.50       115
          2       0.86      0.81      0.83       394

avg / total       0.77      0.75      0.76       509

Loading training set ...
actin_dots_junctions_training.npz (302, 617)
Scoring cells 1 ...
Elapsed time is 1.21097803116 seconds.
stratified 5 fold CV 0.708306010929 0.0456860014218
             precision    recall  f1-score   support

          1       0.62      0.67      0.65       120
          2       0.77      0.74      0.75       182

avg / total       0.71      0.71      0.71       302

Loading training set ...
anaphase_telophase_training.npz (1840, 617)
Scoring cells 1 ...
Elapsed time is 11.2295789719 seconds.
stratified 5 fold CV 0.923981953968 0.0349920292436
             precision    recall  f1-

In [7]:
#logistic regression 1 vs all class weight auto
classifier = "grid_search.GridSearchCV(linear_model.LogisticRegression(penalty='l2',solver='liblinear', multi_class='ovr', class_weight='auto'),param_grid={'C': [2**-5, 2**-4, 2**-3, 2**-2, 2**-1, 2**0, 2**1, 2**2, 2**3, 2**4, 2**5, 2**6]},cv=5,n_jobs=1)" 
main(classifier)

Loading training set ...
actin_blebs_training.npz (509, 617)
Scoring cells 1 ...
Elapsed time is 97.8326058388 seconds.
stratified 5 fold CV 0.746456998641 0.0387391250179
             precision    recall  f1-score   support

          1       0.45      0.59      0.51       115
          2       0.87      0.79      0.83       394

avg / total       0.78      0.75      0.76       509

Loading training set ...
actin_dots_junctions_training.npz (302, 617)
Scoring cells 1 ...
Elapsed time is 40.0054600239 seconds.
stratified 5 fold CV 0.714918032787 0.0477426787377
             precision    recall  f1-score   support

          1       0.62      0.72      0.67       120
          2       0.79      0.71      0.75       182

avg / total       0.73      0.72      0.72       302

Loading training set ...
anaphase_telophase_training.npz (1840, 617)
Scoring cells 1 ...
Elapsed time is 420.642338991 seconds.
stratified 5 fold CV 0.906611181101 0.0487009296926
             precision    recall  f1-

In [62]:
#logistic regression 1 vs all class weight auto, no search
classifier = "linear_model.LogisticRegression(penalty='l2',solver='liblinear', multi_class='ovr', class_weight='auto')" 
main(classifier)

Loading training set ...
actin_blebs_training.npz (509, 617)
Scoring cells 1 ...
Elapsed time is 3.23333287239 seconds.
stratified 5 fold CV 0.734711706465 0.0309570399496
             precision    recall  f1-score   support

          1       0.43      0.57      0.49       115
          2       0.86      0.78      0.82       394

avg / total       0.76      0.73      0.75       509

Loading training set ...
actin_dots_junctions_training.npz (302, 617)
Scoring cells 1 ...
Elapsed time is 0.947407960892 seconds.
stratified 5 fold CV 0.708360655738 0.0360521584588
             precision    recall  f1-score   support

          1       0.62      0.67      0.65       120
          2       0.77      0.74      0.75       182

avg / total       0.71      0.71      0.71       302

Loading training set ...
anaphase_telophase_training.npz (1840, 617)
Scoring cells 1 ...
Elapsed time is 13.0575060844 seconds.
stratified 5 fold CV 0.900638819621 0.0455897228787
             precision    recall  f1

In [56]:
#logistic regression multinomial, lbfgs solver, class weight auto
classifier = "grid_search.GridSearchCV(linear_model.LogisticRegression(penalty='l2',solver='lbfgs', multi_class='multinomial', class_weight='auto'),param_grid={'C': [2**-5, 2**-4, 2**-3, 2**-2, 2**-1, 2**0, 2**1, 2**2, 2**3, 2**4, 2**5, 2**6]},cv=5,n_jobs=1)"
main(classifier)

Loading training set ...
actin_blebs_training.npz (509, 617)
Scoring cells 1 ...
Elapsed time is 18.2757089138 seconds.
stratified 5 fold CV 0.734731120171 0.0354068106022
             precision    recall  f1-score   support

          1       0.43      0.56      0.49       115
          2       0.86      0.79      0.82       394

avg / total       0.76      0.73      0.75       509

Loading training set ...
actin_dots_junctions_training.npz (302, 617)
Scoring cells 1 ...
Elapsed time is 15.8401558399 seconds.
stratified 5 fold CV 0.691639344262 0.0606576083993
             precision    recall  f1-score   support

          1       0.61      0.61      0.61       120
          2       0.74      0.75      0.75       182

avg / total       0.69      0.69      0.69       302

Loading training set ...
anaphase_telophase_training.npz (1840, 617)
Scoring cells 1 ...
Elapsed time is 41.7181739807 seconds.
stratified 5 fold CV 0.888045677489 0.0377241688115
             precision    recall  f1-

In [ ]:
#logistic regression 1 vs all, liblinear solver, class weight auto, l1 penalty
classifier = "grid_search.GridSearchCV(linear_model.LogisticRegression(penalty='l1',solver='liblinear', multi_class='ovr', class_weight='auto'),param_grid={'C': [2**-5, 2**-4, 2**-3, 2**-2, 2**-1, 2**0, 2**1, 2**2, 2**3, 2**4, 2**5, 2**6]},cv=5,n_jobs=1)"
main(classifier)

In [67]:
#KNN 5
classifier = "neighbors.KNeighborsClassifier(n_neighbors=5, weights='distance', algorithm='auto')" 
main(classifier)

Loading training set ...
actin_blebs_training.npz (509, 617)
Scoring cells 1 ...
Elapsed time is 0.134249925613 seconds.
stratified 5 fold CV 0.764259367113 0.0365742887339
             precision    recall  f1-score   support

          1       0.47      0.36      0.41       115
          2       0.82      0.88      0.85       394

avg / total       0.74      0.76      0.75       509

Loading training set ...
actin_dots_junctions_training.npz (302, 617)
Scoring cells 1 ...
Elapsed time is 0.0598161220551 seconds.
stratified 5 fold CV 0.631803278689 0.0820022875776
             precision    recall  f1-score   support

          1       0.53      0.61      0.57       120
          2       0.72      0.65      0.68       182

avg / total       0.64      0.63      0.64       302

Loading training set ...
anaphase_telophase_training.npz (1840, 617)
Scoring cells 1 ...
Elapsed time is 1.08374190331 seconds.
stratified 5 fold CV 0.877267653141 0.0432169697735
             precision    recall  

## fastgentleboosting

In [54]:
#run to define main function which loads training data from local .npy files and does CV
import os
import sys
sys.path.insert(1, '/Users/jyhung/Documents/CellProfiler-Analyst_myversion/cpa/')
sys.path.insert(1, '/Users/jyhung/Documents/CellProfiler-Analyst_myversion/')
from dbconnect import *
import threading
from trainingset import TrainingSet
import fastgentleboosting, multiclasssql
from sklearn import cross_validation, metrics
from sklearn import ensemble, naive_bayes, grid_search, svm, lda, qda, tree, multiclass, linear_model, neighbors

directory = '/Users/jyhung/work/projects/cpamulti/Jones_PNAS_2009/'

def fgb(num_learners):
    classifier = fastgentleboosting.FastGentleBoosting()
    
    for dirname, dirnames, filenames in os.walk(directory):
        for filename in sorted(filenames):
            
            if filename.endswith('.npz'):
                print 'Loading training set ...'
                trainingSet=np.load(directory+filename)
                labels=trainingSet['labels']
                values=trainingSet['values']
                print filename, values.shape
                #print labels, values
                #%time classifier.XValidate(labels, values, folds=20, stratified=True)
                
                # convert labels to a matrix with +1/-1 values only (+1 in the column matching the label, 1-indexed)
                num_classes = max(labels)
                num_samples = len(labels)
                
                label_matrix = -np.ones((len(labels), num_classes), np.int32)
                #print zip(range(len(labels)), np.array(labels)-1)
                for i, j in zip(range(len(labels)), np.array(labels)-1):
                    label_matrix[i, j] = 1
                
                #print labels
                #labels = [x-1 for x in labels]
                #print labels
                #print label_matrix
                
                tic()
                print 'Scoring cells 1 ...'
                folds = 5
                scores = np.zeros(folds)
                total_labels = np.zeros(num_samples)
                colnames = np.array(range(values.shape[1]))
                skf = cross_validation.StratifiedKFold(labels, n_folds=folds)
                for i, (train_index, test_index) in enumerate(skf):
                    #print train_index, test_index
                    values_train, values_test = values[train_index], values[test_index]
                    labels_train, labels_test = label_matrix[train_index], label_matrix[test_index]
                    predicted_labels = classifier.Train(colnames, num_learners, labels_train, values_train, test_values=values_test)
                    #print len(predicted_labels), len(predicted_labels[0])
                    predicted_labels = predicted_labels[-1]
                    total_labels[test_index] = total_labels[test_index] + predicted_labels
                    #print label_matrix[test_index, :].argmax(axis=1)
                    #print predicted_labels
                    scores[i] = metrics.accuracy_score(label_matrix[test_index, :].argmax(axis=1), predicted_labels)
                toc()
                print 'stratified 5 fold CV', scores.mean(), scores.std()
                print metrics.classification_report(label_matrix.argmax(axis=1), total_labels)




In [55]:
fgb(50)

Loading training set ...
actin_blebs_training.npz (509, 617)
Scoring cells 1 ...
Elapsed time is 32.0194680691 seconds.
stratified 5 fold CV 0.821219180742 0.0742454187269
             precision    recall  f1-score   support

          0       0.62      0.55      0.58       115
          1       0.87      0.90      0.89       394

avg / total       0.81      0.82      0.82       509

Loading training set ...
actin_dots_junctions_training.npz (302, 617)
Scoring cells 1 ...
Elapsed time is 24.1298391819 seconds.
stratified 5 fold CV 0.797704918033 0.0596771850779
             precision    recall  f1-score   support

          0       0.72      0.80      0.76       120
          1       0.86      0.80      0.83       182

avg / total       0.80      0.80      0.80       302

Loading training set ...
anaphase_telophase_training.npz (1840, 617)
Scoring cells 1 ...
Elapsed time is 73.0959849358 seconds.
stratified 5 fold CV 0.943023016609 0.042938750544
             precision    recall  f1-s